In [1]:
from core import models
from core import data 
import utils 
import torch.nn as nn
import torch
import os
from tqdm import trange, tqdm
import numpy as np

In [2]:
data_dir = "../../different_cards"
device = "cpu"
exp_dir = "../../classifier_test"

utils.prepare_directory(exp_dir)

In [3]:
logger_titles = [
        'lr',
        'train_loss',
        'val_loss',
        'train_top1',
        'val_top1',
        'train_time',
        'val_time']


In [4]:
criterion = nn.CrossEntropyLoss().to(device)
datahandler = data.CardDataHandler(data_dir, uniform_sampling=True)

13 validation split from training
115 training remains


In [5]:
trainloader, valloader = datahandler.get_dataloaders(batch_size=8)

created uniform sampler


In [6]:
# create the convnet and optimizer
model = models.ConvNet()

lr = 0.0001
momentum = 0.9
wd = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=wd)

In [7]:
log_dir = os.path.join(exp_dir, "training_logs")
utils.prepare_directory(log_dir)
logger = utils.Logger(log_dir, 'logs', logger_titles)

In [8]:
#training loop
n_epochs = 10

best_top1 = 0 
best_confmat = None 
for epoch in trange(0, n_epochs, desc="epoch_monitor", dynamic_ncols=False):
    train_loss, train_top1, train_time = utils.train(model, trainloader, optimizer, criterion, device)
    val_loss, val_top1, confmat, val_time = utils.evaluate(model, valloader, criterion, device)
    
    is_best = best_top1 < val_top1
    if is_best:
        best_top1 = val_top1
        best_confmat = confmat
        
    utils.save_checkpoint({
        'model': model.state_dict(),
        'epoch': epoch, 
        'best_top1':best_top1,
        'val_top1':val_top1}, 
        exp_dir,
        is_best)
    
    logger.append(
            [lr, train_loss, val_loss,
            train_top1, val_top1,  
            train_time, val_time],
            epoch)

    print("train loss: {:.2f}  val loss: {:.2f} val acc: {:.2f}".format(train_loss, val_loss, val_top1))

logger.close()
    

training loop:   0%|          | 0/115 [00:00<?, ?it/s]/Users/jasonzhang/opt/anaconda3/envs/taskTohsakaTorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

training loop: 100%|██████████| 115/115 [00:23<00:00,  4.92it/s]

epoch_monitor:  10%|█         | 1/10 [00:36<05:28, 36.48s/it]

train loss: 1.15  val loss: 0.98 val acc: 0.77



training loop: 100%|██████████| 115/115 [00:21<00:00,  5.34it/s]

epoch_monitor:  20%|██        | 2/10 [01:11<04:45, 35.64s/it]

train loss: 0.94  val loss: 0.85 val acc: 0.54



training loop: 100%|██████████| 115/115 [00:22<00:00,  5.18it/s]

epoch_monitor:  30%|███       | 3/10 [01:47<04:12, 36.01s/it]

...

train loss: 0.26  val loss: 0.27 val acc: 1.00



training loop: 100%|██████████| 115/115 [00:21<00:00,  5.29it/s]

epoch_monitor:  90%|█████████ | 9/10 [05:20<00:35, 35.29s/it]

train loss: 0.28  val loss: 0.23 val acc: 1.00



training loop: 100%|██████████| 115/115 [00:21<00:00,  5.29it/s]

epoch_monitor: 100%|██████████| 10/10 [05:54<00:00, 35.49s/it]


train loss: 0.27  val loss: 0.20 val acc: 1.00


In [9]:
confmat

array([[4, 0, 0],
       [0, 2, 0],
       [0, 0, 7]])

In [10]:
train_top1

0.9739130434782609

In [4]:
model

ConvNet(
  (extractor): Extractor(
    (extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
    

In [30]:
class DimCheck():
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): 
        print(input)
        print(len(input))
        print(input[0])
        print(type(input[0]))
        print("input shape: {}".format(input[0].size()))
        print("output shape: {}".format(output[0].size()))
    def remove(self):
        self.hook.remove() 

In [34]:
for image, label in trainloader:
        image = image.to(device)
        label = label.to(device, non_blocking=True)
        
        output1 = model.extractor(image)
        
        print(output1.shape)
        break

(tensor([[[[0.0000, 0.9823, 1.0330, 1.3835],
          [0.0000, 0.6851, 0.2745, 1.3218],
          [0.0000, 0.0000, 0.1472, 0.4133],
          [0.0000, 0.0000, 0.3473, 0.3521]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.6740, 0.3269, 0.3486, 1.0992],
          [0.6895, 1.2781, 1.5976, 4.5500]],

         [[0.0510, 0.6375, 0.0000, 0.1581],
          [0.7643, 2.0497, 0.0902, 0.0000],
          [0.4224, 0.2055, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         ...,

         [[0.0000, 0.3856, 0.0000, 0.1156],
          [0.0000, 0.9176, 0.9981, 0.8424],
          [0.1459, 0.7396, 1.1426, 0.7488],
          [0.0000, 0.6087, 1.0331, 1.4885]],

         [[0.0000, 0.3155, 0.0000, 0.3677],
          [0.0000, 0.0000, 0.0000, 1.1952],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.4405]],

         [[0.0000, 0.1439, 0.3486, 0.9290],
          [0.0000, 0.0000, 0.0000, 0.1002],
      

In [32]:
avg2dfinal_layer = model.extractor.extractor[8]
print(avg2dfinal_layer)

AdaptiveAvgPool2d(output_size=(1, 1))


In [33]:
_ = DimCheck(avg2dfinal_layer)

In [31]:
_.remove()